In [ ]:
!pip install autokeras
!pip install tensorboardX
!pip install mosaicml
import tensorflow as tf
from keras.datasets import cifar10
import autokeras as ak
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import composer.functional as cf

AttributeError: ignored

In [ ]:
from autokeras.engine.tuner import AutoTuner


AttributeError: ignored

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Take only 5000 samples from the training set
num_samples = 5000
train_images = train_images[:num_samples]
train_labels = train_labels[:num_samples]

# Take only 5000 samples from the test set
test_images = test_images[:num_samples]
test_labels = test_labels[:num_samples]

# The 'train_images', 'train_labels', 'test_images', and 'test_labels' now contain only 5000 samples each
# Normalize the pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0


170498071/170498071 [==============================] - 14s 0us/step


In [ ]:
import autokeras as ak
import numpy as np

class LayerBlock(ak.Block):
    def build(self, hp, inputs=None):
        input_node = tf.nest.flatten(inputs)[0]

        # First Dense Layer
        layer1 = tf.keras.layers.Dense(units=hp.Int("Layer_1", min_value=32, max_value=1024, step=32),activation=hp.Choice("activation_1", values=["relu","softmax", "tanh","sigmoid"]))
        output_node1 = layer1(input_node)

        # Second Dense Layer
        layer2 = tf.keras.layers.Dense(units=hp.Int("Layer_2", min_value=32, max_value=1024, step=32),activation=hp.Choice("activation_2", values=["relu","softmax", "tanh","sigmoid"]))
        output_node2 = layer2(output_node1)

        return output_node2

AttributeError: ignored

In [ ]:
'''import autokeras as ak

class LayerBlock(ak.Block):

  def build(self, hp, inputs=None):
    input_node = tf.nest.flatten(inputs)[0]

    # Tune number of layers
    num_layers = hp.Int('num_layers', 1, 6)

    x = input_node
    for i in range(num_layers):
      # Add dense layer
      layer = tf.keras.layers.Dense(
        units=hp.Int(f'units_{i}', 32, 512, step=32),
        activation=hp.Choice(f'activation_{i}', ["linear","relu", "leaky_relu", "tanh","sigmoid", "softmax", "softplus", "elu", "selu"])
      )
      x = layer(x)

    return x'''

'import autokeras as ak\n\nclass LayerBlock(ak.Block):\n\n  def build(self, hp, inputs=None):\n    input_node = tf.nest.flatten(inputs)[0]\n\n    # Tune number of layers\n    num_layers = hp.Int(\'num_layers\', 1, 6)\n\n    x = input_node\n    for i in range(num_layers):\n      # Add dense layer\n      layer = tf.keras.layers.Dense(\n        units=hp.Int(f\'units_{i}\', 32, 512, step=32),\n        activation=hp.Choice(f\'activation_{i}\', ["linear","relu", "leaky_relu", "tanh","sigmoid", "softmax", "softplus", "elu", "selu"])\n      )\n      x = layer(x)\n\n    return x'

In [ ]:

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="bayesian",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,  # If True, overwrites the results of previous runs.
    max_trials=50,   # Maximum number of different Keras Models to try.
    seed=42,         # Random seed for reproducibility.
    directory="/my/directory",  # Directory to store the results.
    project_name='my_custom_image_classifier',
    objective="val_loss"  # Metric to optimize during the search.
)
cf.apply_blurpool(auto_model)
cf.apply_squeeze_excite(auto_model)

# Train the model
auto_model.fit(train_images, train_labels, epochs=1)
print(auto_model.evaluate(test_images, test_labels))


AttributeError: ignored

In [ ]:
tuner = auto_model.tuner
trial_ids = tuner.oracle.trials
trials_hparams=[]
score=[]
for trial_id in trial_ids:
    trial = auto_model.tuner.oracle.get_trial(trial_id)
    trial.best_step
    trial.metrics
    hparams = trial.hyperparameters.values
    score.append(trial.score)
  # Append to list
    trials_hparams.append(hparams)


import pandas as pd
df_bayesian = pd.DataFrame(trials_hparams)
df_bayesian['val_Loss']=score
df_bayesian

In [ ]:

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="random",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,  # If True, overwrites the results of previous runs.
    max_trials=50,   # Maximum number of different Keras Models to try.
    seed=42,         # Random seed for reproducibility.
    directory="/my/directory",  # Directory to store the results.
    project_name='my_custom_image_classifier',
    objective="val_loss"  # Metric to optimize during the search.
)


# Train the model
auto_model.fit(train_images, train_labels, epochs=1)
print(auto_model.evaluate(test_images, test_labels))


In [ ]:
tuner = auto_model.tuner
trial_ids = tuner.oracle.trials
trials_hparams=[]
score=[]
for trial_id in trial_ids:
    trial = auto_model.tuner.oracle.get_trial(trial_id)
    trial.best_step
    trial.metrics
    hparams = trial.hyperparameters.values
    score.append(trial.score)
  # Append to list
    trials_hparams.append(hparams)


import pandas as pd
df_random = pd.DataFrame(trials_hparams)
df_random['val_Loss']=score
df_random

In [ ]:

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="greedy",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,  # If True, overwrites the results of previous runs.
    max_trials=50,   # Maximum number of different Keras Models to try.
    seed=42,         # Random seed for reproducibility.
    directory="/my/directory",  # Directory to store the results.
    project_name='my_custom_image_classifier',
    objective="val_loss"  # Metric to optimize during the search.
)


# Train the model
auto_model.fit(train_images, train_labels, epochs=1)
print(auto_model.evaluate(test_images, test_labels))


In [ ]:
tuner = auto_model.tuner
trial_ids = tuner.oracle.trials
trials_hparams=[]
score=[]
for trial_id in trial_ids:
    trial = auto_model.tuner.oracle.get_trial(trial_id)
    trial.best_step
    trial.metrics
    hparams = trial.hyperparameters.values
    score.append(trial.score)
  # Append to list
    trials_hparams.append(hparams)


import pandas as pd
df_greedy = pd.DataFrame(trials_hparams)
df_greedy['val_Loss']=score
df_greedy

In [ ]:

input_node = ak.Input()
output_node = LayerBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

# Create the AutoModel
auto_model = ak.AutoModel(
    tuner="hyperband",
    inputs=input_node,
    outputs=output_node,
    overwrite=True,  # If True, overwrites the results of previous runs.
    max_trials=50,   # Maximum number of different Keras Models to try.
    seed=42,         # Random seed for reproducibility.
    directory="/my/directory",  # Directory to store the results.
    project_name='my_custom_image_classifier',
    objective="val_loss"  # Metric to optimize during the search.
)


# Train the model
auto_model.fit(train_images, train_labels, epochs=1)
print(auto_model.evaluate(test_images, test_labels))


In [ ]:
tuner = auto_model.tuner
trial_ids = tuner.oracle.trials
trials_hparams=[]
score=[]
for trial_id in trial_ids:
    trial = auto_model.tuner.oracle.get_trial(trial_id)
    trial.best_step
    trial.metrics
    hparams = trial.hyperparameters.values
    score.append(trial.score)
  # Append to list
    trials_hparams.append(hparams)


import pandas as pd
df_hyperband = pd.DataFrame(trials_hparams)
df_hyperband['val_Loss']=score
df_hyperband

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_bayesian.to_csv("/content/drive/MyDrive/Capsotn3/Capstone/Bayesian1.csv")
df_random.to_csv("/content/drive/MyDrive/Capsotn3/Capstone/Random1.csv")
df_greedy.to_csv("/content/drive/MyDrive/Capsotn3/Capstone/greedy1.csv")
df_hyperband.to_csv("/content/drive/MyDrive/Capsotn3/Capstone/hyperband1.csv")

In [ ]:
import matplotlib.pyplot as plt

# Your data
x = df.index
y1 = df['layer_block_1/Layer_1']
y2 = df['val_Loss']

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.scatter(x, y1, label='Layer_1 units', color='b')
ax1.set_xlabel('Index')
ax1.set_ylabel('Layer_1 units', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis
ax2.plot(x, y2, label='Val_loss', color='r')
ax2.set_ylabel('Val_loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2


# Show the plot
plt.grid(True)
plt.title('Dual Y-Axis Plot')
plt.show()


multiple layers

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['val_Loss'], c='blue', marker='o', label='Loss')  # Corrected column name
plt.xlabel('Configuration Index')
plt.ylabel('Validation Loss')
plt.title('Hyperparameter Search Results')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Your data
x = df.index
y1 = df['layer_block_1/num_layers']
y2 = df['val_Loss']

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.scatter(x, y1, label='Layer_1 units', color='b')
ax1.set_xlabel('Index')
ax1.set_ylabel('Layer_1 units', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis
ax2.plot(x, y2, label='Val_loss', color='r')
ax2.set_ylabel('Val_loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2


# Show the plot
plt.grid(True)
plt.title('Dual Y-Axis Plot')
plt.show()


In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt

# Your data
x = df.index
y1 = df['layer_block_1/units_0']
y2 = df['val_Loss']

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.scatter(x, y1, label='Layer_1 units', color='b')
ax1.set_xlabel('Index')
ax1.set_ylabel('Layer_1 units', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis
ax2.plot(x, y2, label='Val_loss', color='r')
ax2.set_ylabel('Val_loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2


# Show the plot
plt.grid(True)
plt.title('Dual Y-Axis Plot')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your data
x = df.index
y1 = df['layer_block_1/units_1']
y2 = df['val_Loss']

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.scatter(x, y1, label='Layer_2 units', color='k')
ax1.set_xlabel('Index')
ax1.set_ylabel('Layer_2 units', color='k')
ax1.tick_params(axis='y', labelcolor='k')

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis
ax2.plot(x, y2, label='Val_loss', color='r')
ax2.set_ylabel('Val_loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2


# Show the plot
plt.grid(True)
plt.title('Dual Y-Axis Plot')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your data
x = df.index
y1 = df['layer_block_1/units_2']
y2 = df['val_Loss']

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.scatter(x, y1, label='Layer_3 units', color='k')
ax1.set_xlabel('Index')
ax1.set_ylabel('Layer_3 units', color='k')
ax1.tick_params(axis='y', labelcolor='k')

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis
ax2.plot(x, y2, label='Val_loss', color='r')
ax2.set_ylabel('Val_loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2


# Show the plot
plt.grid(True)
plt.title('Dual Y-Axis Plot')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your data
x = df.index
y1 = df['layer_block_1/units_3']
y2 = df['val_Loss']

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.scatter(x, y1, label='Layer_4 units', color='k')
ax1.set_xlabel('Index')
ax1.set_ylabel('Layer_4 units', color='k')
ax1.tick_params(axis='y', labelcolor='k')

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis
ax2.plot(x, y2, label='Val_loss', color='r')
ax2.set_ylabel('Val_loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2


# Show the plot
plt.grid(True)
plt.title('Dual Y-Axis Plot')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Your data
x = df.index
y1 = df['layer_block_1/units_3']
y2 = df['val_Loss']

# Create a figure and axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the primary y-axis
ax1.scatter(x, y1, label='Layer_4 units', color='k')
ax1.set_xlabel('Index')
ax1.set_ylabel('Layer_4 units', color='k')
ax1.tick_params(axis='y', labelcolor='k')

# Create a secondary y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the secondary y-axis
ax2.plot(x, y2, label='Val_loss', color='r')
ax2.set_ylabel('Val_loss', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines = lines1 + lines2
labels = labels1 + labels2


# Show the plot
plt.grid(True)
plt.title('Dual Y-Axis Plot')
plt.show()


In [ ]:
df

Old results

In [ ]:
import pandas as pd

first_row = df.loc[df['val_Loss'].argmin()]
# Select numeric columns
numeric_cols = df.select_dtypes(include=['integer', 'float']).columns

# Calculate similarity of each row vs first row
from scipy.spatial.distance import cosine

similarity = df.apply(lambda x: 1 - cosine(x[numeric_cols], first_row[numeric_cols]), axis=1)

# Add similarities as new column
df['Cosine_similarity'] = similarity


# Create a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(df['Cosine_similarity'], df['val_Loss'], marker='o', c='b', label='Data Points')

# Label the axes and add a legend
plt.xlabel('Similarity to Best Architechture')
plt.ylabel('Validation Loss')
plt.legend()

# Show the plot
plt.title('Scatter Plot of Similarity(Cosine_similarity) vs. Validation Loss')
plt.grid(True)
plt.show()



In [ ]:
from scipy.spatial.distance import euclidean

# Calculate Euclidean distance instead of cosine similarity
similarity = df.apply(lambda x: 1/(1 + euclidean(x[numeric_cols], first_row[numeric_cols])), axis=1)

# Invert distance to get similarity
similarity = 1/similarity

# Add similarities as new column
df['euclidean_distance'] = similarity


# Create a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(df['euclidean_distance'], df['val_Loss'], marker='o', c='b', label='Data Points')

# Label the axes and add a legend
plt.xlabel('Similarity to Best Architechture')
plt.ylabel('Validation Loss')
plt.legend()

# Show the plot
plt.title('Scatter Plot of Similarity(euclidean_distance) vs. Validation Loss')
plt.grid(True)
plt.show()

In [ ]:
df